In [ ]:
# 1. First, define your "meanings" or "classes"
# This map turns a human-readable label into an integer ID.
LABEL_MAP = {
    "Financial": 0,
    "River": 1
}

# 2. Then, create your list of data points.
# Each line is a tuple: (sentence, target_word_index, label)
data = [
    # "I went to the bank to deposit money"
    #  0   1    2   3    4   5    6       7
    # "bank" is at index 4
    ("I went to the bank to deposit money", 4, LABEL_MAP["Financial"]),

    # "He sat on the river bank"
    #  0   1   2   3     4     5
    # "bank" is at index 5
    ("He sat on the river bank", 5, LABEL_MAP["River"]),

    # "The savings bank is closed today"
    #  0     1       2    3    4      5
    # "bank" is at index 2
    ("The savings bank is closed today", 2, LABEL_MAP["Financial"]),

    # "The boat is near the bank"
    #  0    1   2   3    4     5
    # "bank" is at index 5
    ("The boat is near the bank", 5, LABEL_MAP["River"]),

    # "I need a loan from the bank"
    #  0  1    2  3    4    5     6
    # "bank" is at index 5
    ("I need a loan from the bank", 5, LABEL_MAP["Financial"]),

    # "The river bank was muddy"
    #  0     1     2    3     4
    # "bank" is at index 2
    ("The river bank was muddy", 2, LABEL_MAP["River"]),
]

# You can print it to see the result
print(data)

[('I went to the bank to deposit money', 4, 0), ('He sat on the river bank', 5, 1), ('The savings bank is closed today', 2, 0), ('The boat is near the bank', 5, 1), ('I need a loan from the bank', 5, 0), ('The river bank was muddy', 2, 1)]


In [ ]:
# 1. Collect all words
all_words = set()
for sentence, index, label in data:
    words_in_sentence = sentence.lower().split() # split() turns the string into a list of words
    all_words.update(words_in_sentence)         # .update() adds all items from the list to the set

# 2. Create the vocabulary mapping
# Start with 0 for the PAD token
vocab = {"<PAD>": 0}
index = 1
for word in sorted(list(all_words)): # sort them so the order is consistent
    vocab[word] = index
    index += 1

print("\nYour vocabulary:")
print(vocab)


Your vocabulary:
{'<PAD>': 0, 'a': 1, 'bank': 2, 'boat': 3, 'closed': 4, 'deposit': 5, 'from': 6, 'he': 7, 'i': 8, 'is': 9, 'loan': 10, 'money': 11, 'muddy': 12, 'near': 13, 'need': 14, 'on': 15, 'river': 16, 'sat': 17, 'savings': 18, 'the': 19, 'to': 20, 'today': 21, 'was': 22, 'went': 23}


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim  # You'll need this for the optimizer
import math